In [200]:
import numpy as np
from itertools import product
import pandas as pd
import os
import foraging_toolkit as ft
import environments
import simulation
import simulation_old 
import success_metrics
import imp
imp.reload(ft)
imp.reload(simulation)
imp.reload(simulation_old)
imp.reload(environments)
imp.reload(success_metrics)


<module 'success_metrics' from 'c:\\Users\\admin\\Dropbox\\Code\\Basis-code\\new-collaborative-intelligence\\mx_refactored\\success_metrics.py'>

Parameters of the forward model. Specify the parameter values for all the cases you are interested in. 

In [201]:
# Parameters of interest in our communication model 
c_trust = np.array([0, 0.5])
sight_radius = [5]
reward_patch_dim = [1, 4]

# List of all possible combination of parameter values 
param_list = [i for i in product(c_trust, sight_radius, reward_patch_dim)]

print(len(param_list))
print('param_list', param_list)




4
param_list [(0.0, 5, 1), (0.0, 5, 4), (0.5, 5, 1), (0.5, 5, 4)]


Generate a dataframe containing all possible combinations of the parameter values specified above.

In [202]:
#each row of both dataframes corresponds to one experiment.  
#columns are parameters 
metadataDF = pd.DataFrame(param_list)

# success_metricsDF = pd.DataFrame(results)
# allresultsDF = pd.concat([metadataDF, success_metricsDF], axis='columns') # concatenate metadata and results 
metadataDF.columns = ["c_trust", "sight_radius", "reward_patch_dim"] 
metadataDF['sim index'] = np.arange(len(metadataDF)).astype(int)
metadataDF.head()
display(metadataDF)

,c_trust,sight_radius,reward_patch_dim,sim index
0,0.0,5,1,0
1,0.0,5,4,1
2,0.5,5,1,2
3,0.5,5,4,3


In [203]:
# select a specific row of the data frame
ind = 1
row = metadataDF.iloc[[ind]]
print(row)
print(row['c_trust'].iloc[0])
print(row['sight_radius'].iloc[0])
print(row['reward_patch_dim'].iloc[0])

   c_trust  sight_radius  reward_patch_dim  sim index
1      0.0             5                 4          1
0.0
5
4


In [204]:
test_arr = np.zeros((5))
print(test_arr.shape)

(5,)


*Part 2*
Test run one of the simulations from metadata (one row)

In [205]:
nsims = len(metadataDF)
N_frames = 50
N_agents = 9
N_total_food_units = 16
edge_size = 30
N_runs = 100
# nsims = 2 # TEST

# data frame to store all success values across all simulations and runs of each simulation
success_measures_allDF = pd.DataFrame() 
success_sims_list = []

home_dir = "C:\\Basis_data\\" # 

# save metadata to home directory
metadataDF.to_csv(home_dir + '\metadataDF' + '.csv')

for si in range(nsims):
    
    # 1. pull out parameters from row si in the metadata
    df_row = metadataDF.iloc[[si]]
    c_trust = df_row['c_trust'].iloc[0]
    sight_radius = df_row['sight_radius'].iloc[0]
    reward_patch_dim = df_row['reward_patch_dim'].iloc[0].astype(int)

    # print(df_row)
    # print(reward_patch_dim)

    mean_times_to_first_reward = np.zeros((N_runs))
    num_birds_failed = np.zeros((N_runs))

    # Do multiple runs of the simulation and store the results in a results dataframe
    for ri in range(N_runs):
        # 2. Feed the parameters through the run() function once (later we'll do this many times)
        env = environments.Environment(edge_size=edge_size, N_total_food_units=N_total_food_units, patch_dim=reward_patch_dim)
        env.add_food_patches() #food_statistics_type="drop_food_once")
        # sim = simulation_old.Simulation(env, N_agents=N_agents, N_frames=N_frames)
        sim = simulation.SimulateCommunicators(env, N_frames, N_agents, c_trust=c_trust, sight_radius=sight_radius)
        sim.run()
        # print(sim.all_birdsDF)
        # # print(sim.all_rewardsDF) 
        # # by this point we should have birdlocsDF and rewardlocsDF 
        

        # # # 3. Compute success measures 
        
        time_to_first_allbirds = np.zeros(N_agents)
        for bird_id in range(1, N_agents + 1):
            singlebirdDF = sim.all_birdsDF.loc[sim.all_birdsDF['bird'] == bird_id]  
            time_to_first_allbirds[bird_id - 1] = success_metrics.compute_time_to_first_reward(singlebirdDF, sim.all_rewardsDF, N_frames)
            
        # these are the success measures
        mean_times_to_first_reward[ri] = np.mean(time_to_first_allbirds) # take the average across birds
        num_birds_failed[ri] = np.sum(time_to_first_allbirds < 0) # number of birds that failed to reach food
        # print(num_birds_failed)
        #

        # SAVE THE RAW DATA 
        # make a new folder  
        sim_folder = 'sim' + str(si) + '_run' + str(ri)
        sim_dir = home_dir + sim_folder
        if not os.path.isdir(sim_dir):
            os.makedirs(sim_dir)
            print("Directory %s was created." %sim_folder)

        birdfile = '\\birdlocsDF'
        rewardfile = '\\rewardlocsDF'
        sim.all_birdsDF.to_csv(sim_dir + birdfile + ".csv")
        sim.all_rewardsDF.to_csv(sim_dir + rewardfile + ".csv")

       

    #4. Combine the metadata and the success measures into the results dataframe 
        # create a new row in the results data frame, then concatenate 
    success_measures_onesim_DF = pd.DataFrame(
        {
            "run index": np.arange(N_runs),
            "time to first food": mean_times_to_first_reward,
            "num birds failed": num_birds_failed,
        }
    )

    success_sims_list.append(success_measures_onesim_DF)

success_measures_allDF = pd.concat(success_sims_list)

    # 5. Save the raw data into a separate folder.


print(len(success_sims_list))
print(len(mean_times_to_first_reward))
print(len(success_measures_onesim_DF))
print(display(success_measures_onesim_DF))
print(display(success_measures_allDF))

# Concatenate metadataDF and success_measures_allDF into a new dataframe called resultsDF

    
# Save the results dataframe into the same folder as the metadata


Directory sim0_run0 was created.
Directory sim0_run1 was created.
Directory sim0_run2 was created.
Directory sim0_run3 was created.
Directory sim0_run4 was created.
Directory sim0_run5 was created.
Directory sim0_run6 was created.
Directory sim0_run7 was created.
Directory sim0_run8 was created.
Directory sim0_run9 was created.
Directory sim0_run10 was created.
Directory sim0_run11 was created.
Directory sim0_run12 was created.
Directory sim0_run13 was created.
Directory sim0_run14 was created.
Directory sim0_run15 was created.
Directory sim0_run16 was created.
Directory sim0_run17 was created.
Directory sim0_run18 was created.
Directory sim0_run19 was created.
Directory sim0_run20 was created.
Directory sim0_run21 was created.
Directory sim0_run22 was created.
Directory sim0_run23 was created.
Directory sim0_run24 was created.
Directory sim0_run25 was created.
Directory sim0_run26 was created.
Directory sim0_run27 was created.
Directory sim0_run28 was created.
Directory sim0_run29 was

,run index,time to first food,num birds failed
0,0,-0.222222,8.0
1,1,5.333333,3.0
2,2,0.777778,7.0
3,3,8.444444,3.0
4,4,5.444444,2.0
...,...,...,...
95,95,4.777778,5.0
96,96,3.777778,1.0
97,97,2.555556,6.0
98,98,5.111111,7.0


None


,run index,time to first food,num birds failed
0,0,2.111111,0.0
1,1,7.111111,0.0
2,2,1.333333,0.0
3,3,1.444444,0.0
4,4,2.000000,0.0
...,...,...,...
95,95,4.777778,5.0
96,96,3.777778,1.0
97,97,2.555556,6.0
98,98,5.111111,7.0


None


In [206]:
# Concatenate metadataDF and success_measures_allDF into a new dataframe called resultsDF

# First, duplicate each row (sim) of metadataDF by the number of runs
newDF = pd.DataFrame(np.repeat(metadataDF.values, N_runs, axis=0))
newDF.columns = metadataDF.columns
print(newDF)




     c_trust  sight_radius  reward_patch_dim  sim index
0        0.0           5.0               1.0        0.0
1        0.0           5.0               1.0        0.0
2        0.0           5.0               1.0        0.0
3        0.0           5.0               1.0        0.0
4        0.0           5.0               1.0        0.0
..       ...           ...               ...        ...
395      0.5           5.0               4.0        3.0
396      0.5           5.0               4.0        3.0
397      0.5           5.0               4.0        3.0
398      0.5           5.0               4.0        3.0
399      0.5           5.0               4.0        3.0

[400 rows x 4 columns]


In [207]:
# change the indexing of the rows of this dataframe so we can concatenate with metadataDF
successDF = success_measures_allDF.reset_index(drop=True)
display(successDF)

,run index,time to first food,num birds failed
0,0,2.111111,0.0
1,1,7.111111,0.0
2,2,1.333333,0.0
3,3,1.444444,0.0
4,4,2.000000,0.0
...,...,...,...
395,95,4.777778,5.0
396,96,3.777778,1.0
397,97,2.555556,6.0
398,98,5.111111,7.0


In [208]:
# Concatenate metadataDF and 
allresultsDF = pd.concat([newDF, successDF], axis=1)
display(allresultsDF)

,c_trust,sight_radius,reward_patch_dim,sim index,run index,time to first food,num birds failed
0,0.0,5.0,1.0,0.0,0,2.111111,0.0
1,0.0,5.0,1.0,0.0,1,7.111111,0.0
2,0.0,5.0,1.0,0.0,2,1.333333,0.0
3,0.0,5.0,1.0,0.0,3,1.444444,0.0
4,0.0,5.0,1.0,0.0,4,2.000000,0.0
...,...,...,...,...,...,...,...
395,0.5,5.0,4.0,3.0,95,4.777778,5.0
396,0.5,5.0,4.0,3.0,96,3.777778,1.0
397,0.5,5.0,4.0,3.0,97,2.555556,6.0
398,0.5,5.0,4.0,3.0,98,5.111111,7.0


*Save data in a folder*


In [209]:
# save metadata to home directory
allresultsDF.to_csv(home_dir + 'resultsDF' + '.csv')

Test one of the simulations by animating it.

In [210]:
communicators = ft.object_from_data(sim.all_birdsDF, sim.all_rewardsDF)
ft.animate_birds(communicators, plot_rewards=True,
                  width = 600, height = 600, point_size = 10)

In [211]:
print(sim.all_birdsDF.loc[sim.all_birdsDF['bird'] == 1])

       x     y  time  bird           type
0   22.0  18.0     1     1  communicators
1   25.0  18.0     2     1  communicators
2   24.0  16.0     3     1  communicators
3   24.0  17.0     4     1  communicators
4   22.0  17.0     5     1  communicators
5   21.0  17.0     6     1  communicators
6   18.0  17.0     7     1  communicators
7   16.0  18.0     8     1  communicators
8   15.0  19.0     9     1  communicators
9   16.0  20.0    10     1  communicators
10  17.0  21.0    11     1  communicators
11  18.0  20.0    12     1  communicators
12  21.0  20.0    13     1  communicators
13  19.0  21.0    14     1  communicators
14  18.0  20.0    15     1  communicators
15  16.0  19.0    16     1  communicators
16  14.0  21.0    17     1  communicators
17  12.0  23.0    18     1  communicators
18  11.0  25.0    19     1  communicators
19  11.0  25.0    20     1  communicators
20  11.0  25.0    21     1  communicators
21  11.0  25.0    22     1  communicators
22  11.0  25.0    23     1  commun

In [212]:
# print(sim.all_rewardsDF[]) 
print(sim.all_rewardsDF.loc[sim.all_rewardsDF['time'] == 4])


     x   y  time
0    9  25     4
1   10  25     4
2   11  25     4
3   12  25     4
4    9  26     4
5   10  26     4
6   11  26     4
7   12  26     4
8    9  27     4
9   10  27     4
10  11  27     4
11  12  27     4
12   9  28     4
13  10  28     4
14  11  28     4
15  12  28     4


In [213]:

time_to_first_allbirds = np.zeros(N_agents)
# print(time_to_first_allbirds.shape)

for bird_id in range(1, N_agents + 1):
    # bird_id = 2
    singlebirdDF = sim.all_birdsDF.loc[sim.all_birdsDF['bird'] == bird_id] 
    display(singlebirdDF)
    time_to_first_allbirds[bird_id - 1] = success_metrics.compute_time_to_first_reward(singlebirdDF, sim.all_rewardsDF, N_frames)

    print(time_to_first_allbirds)

num_birds_failed = np.sum(time_to_first_allbirds < 0)
print('Num birds failed to find food = ' + str(num_birds_failed))


,x,y,time,bird,type
0,22.0,18.0,1,1,communicators
1,25.0,18.0,2,1,communicators
2,24.0,16.0,3,1,communicators
3,24.0,17.0,4,1,communicators
4,22.0,17.0,5,1,communicators
5,21.0,17.0,6,1,communicators
6,18.0,17.0,7,1,communicators
7,16.0,18.0,8,1,communicators
8,15.0,19.0,9,1,communicators
9,16.0,20.0,10,1,communicators


[18.  0.  0.  0.  0.  0.  0.  0.  0.]


,x,y,time,bird,type
0,1.0,10.0,1,2,communicators
1,1.0,9.0,2,2,communicators
2,0.0,7.0,3,2,communicators
3,2.0,9.0,4,2,communicators
4,1.0,10.0,5,2,communicators
5,3.0,8.0,6,2,communicators
6,1.0,10.0,7,2,communicators
7,0.0,9.0,8,2,communicators
8,2.0,11.0,9,2,communicators
9,0.0,12.0,10,2,communicators


[18. -1.  0.  0.  0.  0.  0.  0.  0.]


,x,y,time,bird,type
0,29.0,24.0,1,3,communicators
1,27.0,25.0,2,3,communicators
2,28.0,24.0,3,3,communicators
3,26.0,23.0,4,3,communicators
4,27.0,22.0,5,3,communicators
5,26.0,23.0,6,3,communicators
6,27.0,25.0,7,3,communicators
7,26.0,25.0,8,3,communicators
8,26.0,26.0,9,3,communicators
9,25.0,27.0,10,3,communicators


...

[18. -1. -1. -1. -1. -1.  1.  0.  0.]


,x,y,time,bird,type
0,16.0,4.0,1,8,communicators
1,17.0,3.0,2,8,communicators
2,15.0,5.0,3,8,communicators
3,17.0,5.0,4,8,communicators
4,18.0,5.0,5,8,communicators
5,20.0,7.0,6,8,communicators
6,20.0,5.0,7,8,communicators
7,19.0,5.0,8,8,communicators
8,17.0,3.0,9,8,communicators
9,16.0,5.0,10,8,communicators


[18. -1. -1. -1. -1. -1.  1. -1.  0.]


,x,y,time,bird,type
0,11.0,12.0,1,9,communicators
1,13.0,13.0,2,9,communicators
2,15.0,13.0,3,9,communicators
3,16.0,14.0,4,9,communicators
4,16.0,16.0,5,9,communicators
5,19.0,16.0,6,9,communicators
6,18.0,16.0,7,9,communicators
7,17.0,18.0,8,9,communicators
8,19.0,16.0,9,9,communicators
9,19.0,15.0,10,9,communicators


[18. -1. -1. -1. -1. -1.  1. -1. -1.]
Num birds failed to find food = 7
